# İş Problemi
Facebook kısa süre önce mevcut "maximumbidding" adı verilen teklif verme türüne
alternatif olarak yeni bir teklif türü olan "average bidding"’i tanıttı. Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test etmeye karar verdi ve average bidding'in maximumbidding'den daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B testi yapmak istiyor. A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden bu A/B testinin sonuçlarını analiz etmenizi bekliyor. Bombabomba.com için nihai başarı ölçütü Purchase'dır. Bu nedenle, istatistiksel testler için Purchase metriğine odaklanılmalıdır.

İlk olarak kütüphanelerimizi ekleyelim ve output ayarlarımızı yapalım.

In [8]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)
pd.set_option("display.width", 180)
pd.set_option("display.float_format", lambda x: "%.5f" % x)

## Görev 1: Veriyi Hazırlama ve Analiz Etme

### Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.

In [10]:
df_c = pd.read_excel("C:/Users/oe/PycharmProjects/measurement_problems/datasets/ab_testing.xlsx",
                     sheet_name="Control Group")
df_t = pd.read_excel("C:/Users/oe/PycharmProjects/measurement_problems/datasets/ab_testing.xlsx",
                     sheet_name="Test Group")

### Adım 2: Kontrol ve test grubu verilerini analiz ediniz.

In [11]:
df_c.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,120512.41176,18807.44871,79033.83492,112691.97077,119291.30077,132050.57893,158605.92048
Click,40.00000,3967.54976,923.09507,1836.62986,3376.81902,3931.35980,4660.49791,6019.69508
Purchase,40.00000,582.10610,161.15251,311.62952,444.62683,551.35573,699.86236,889.91046
Earning,40.00000,2514.89073,282.73085,1939.61124,2280.53743,2544.66611,2761.54540,3171.48971


In [12]:
df_t.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,120512.41176,18807.44871,79033.83492,112691.97077,119291.30077,132050.57893,158605.92048
Click,40.00000,3967.54976,923.09507,1836.62986,3376.81902,3931.35980,4660.49791,6019.69508
Purchase,40.00000,582.10610,161.15251,311.62952,444.62683,551.35573,699.86236,889.91046
Earning,40.00000,2514.89073,282.73085,1939.61124,2280.53743,2544.66611,2761.54540,3171.48971


### Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

In [13]:
df_c["group"] = "control"
df_t["group"] = "test"

df = pd.concat([df_c, df_t], axis=0, ignore_index=False)

## Görev 2: A/B Testinin Hipotezinin Tanımlanması

### Adım 1: Hipotezi Tanımlayınız.
H0: M1 = M2 Average Bidding ve Maximum Bidding arasında istatistiksel olarak anlamlı bir farklılık yoktur.

H1: M1 != M2 Average Bidding ve Maximum Bidding arasında istatistiksel olarak anlamlı bir farklılık vardır.

### Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.

In [14]:
df.groupby("group").agg({"Purchase": "mean"})

,Purchase
group,
control,550.89406
test,582.10610


Group'a göre Purchase ortalamasına bakıldığında test grubundakiler daha çok kazanç getirmiş. Ancak bu şans eseri olmuş olabilir.

## Görev 3: Hipotez Testinin Gerçekleştirilmesi

### Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız. 
### Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. 
### Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni üzerinden ayrı ayrı test ediniz.

### Normallik Varsayımı 

In [17]:
test_stat, pvalue = shapiro(df.loc[df["group"] == "control", "Purchase"])
print("Test Stat = %.4f, p-value= %.4f" % (test_stat, pvalue))

Test Stat = 0.9773, p-value= 0.5891


p-value değerine bakıldığında 0.05'ten büyük olduğu görülüyor. Bu durumda H0 reddedilemez. Control grubu için normallik varsayımı sağlanmaktadır.

In [22]:
test_stat, pvalue = shapiro(df.loc[df["group"] == "test", "Purchase"])
print("Test Stat = %.4f, p-value= %.4f" % (test_stat, pvalue))

Test Stat = 0.9589, p-value= 0.1541


p-value değerine bakıldığında 0.05'ten büyük olduğu görülüyor. Bu durumda H0 reddedilemez. Test grubu için de normallik varsayımı sağlanmaktadır.

In [21]:
test_stat, pvalue = levene(df.loc[df["group"] == "control", "Purchase"],
                           df.loc[df["group"] == "test", "Purchase"])
print("Test Stat = %.4f, p-value= %.4f" % (test_stat, pvalue))

Test Stat = 2.6393, p-value= 0.1083


p-value değerine bakıldığında 0.05'ten büyük olduğu görülüyor. Bu durumda H0 reddedilemez. Varyanslar homojendir.

### Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

Normallik ve varyans homojenliği sağlandığı için t testi uygulayacağız.

In [23]:
test_stat, pvalue = ttest_ind(df.loc[df["group"] == "control", "Purchase"],
                           df.loc[df["group"] == "test", "Purchase"],
                           equal_var=True)
print("Test Stat = %.4f, p-value= %.4f" % (test_stat, pvalue))

Test Stat = -0.9416, p-value= 0.3493


"equal_var=True" olma sebebi varyans homojenliğinin sağlanmış olmasından kaynaklanıyor. Eğer ki varyanslar homojen dağılmamış olsaydı "equal_var=False" demek zorundaydık.

p-value değeri 0.05'ten büyüktür. Bu durumda H0 hipotezi reddedilemez.

### Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiksel olarak anlamlı bir fark olup olmadığını yorumlayınız.

İlk olarak normallik varsayımı sonrasında ise varyans homojenliği ölçülmüştür ve H0 hipotezi reddedilememiş ve varyans homojenliği olduğu görülmüştür. Buna uygun olarak değişkenlere T-Testi uygulanmıştır, Kontrol ve Test grubu satın alma ortalamaları arasında istatistiksel olarak anlamlı bir fark bulunmamıştır. Yani başta belirlemiş olduğumuz ortalama farkı şans eseri ortaya çıkmıştır ve istatistiksel olarak fark yoktur.

## Görev 4: Sonuçların Analizi

### Adım 1: Hangi testi kullandınız, sebeplerini belirtiniz.

Birbirinden bağımsız iki durum olduğu için AB testi uygulamam gerekiyordu bu yüzden hipotezlerimi oluşturdum. Sonrasında normallik varsayımı ve varyans homojenliğini kontrol ettim. Bu kontroller sonucu H0 hipotezimin reddedilemediğini gördüm. Bu sebeple son olarak parametrik test olan Bağımsız İki Örneklem T-Testini uyguladım.

### Adım 2: Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz.

Elde etmiş olduğum test sonuçlarına göre iki örneklem arasında istatistiksel olarak fark yoktur. Bu nedenle average bidding'e geçiş yapmanın lüzumu yoktur. Maximum bidding ile devam edilebilir.